<a href="https://colab.research.google.com/github/Shridat/Insuarance-policy-RAG-chatbot/blob/main/RAG_based_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain huggingface_hub transformers sentence_transformers pdfplumber langchain-community faiss-cpu rouge-score sacrebleu ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 30.1 M

In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer, util
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub, HuggingFacePipeline
from langchain.prompts import PromptTemplate
import pandas as pd
from rouge_score import rouge_scorer
import numpy as np
import sacrebleu
import re
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)
from datasets import Dataset
from langchain.chains import RetrievalQA
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.question_answering import load_qa_chain

# Text Extraction Step

In [6]:
#extract text from the pdf using pdfplumber
import pdfplumber

def extract_pdf(pdf_path):
    text = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

text = extract_pdf('policy-booklet-0923.pdf')
print(text[:2000])
print(len(text))

Your car insurance
policy bookletWelcome to Churchill
This booklet tells you about your car insurance
About the policy Words in bold type
The policy is made up of: Some of the words and phrases we use in
> This booklet. this booklet have a specific meaning – for
> Your car insurance details. example, your car or modifications.
> Your certificate (or certificates) We’ve highlighted these words using bold
type. You can find the exact meanings of
of motor insurance.
these words in the ‘Glossary’ on page 4,
If the policy includes Green Flag breakdown cover:
or at the start of each section.
> Your breakdown cover and your car
insurance are part of the same policy. Comprehensive with DriveSure
> The policy also includes the Green Flag If you have a Comprehensive with
policy booklet we’ve given you. DriveSure policy:
If you have a policy that includes DriveSure: > Your cover is the same as a
> The policy also includes the DriveSure terms Comprehensive policy.
and conditions we’ve given you. >

In [7]:
def clean_text(text):
    # Remove special characters and multiple new lines
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[^A-Za-z0-9,.?!\s]', '', text)  # Remove non-alphanumeric characters
    return text

#text = clean_text(text)
#print(text[:2000])

In [8]:
#Divide extracted long text into equal chunks
text_splitter = CharacterTextSplitter(separator = '\n', chunk_size=1000, chunk_overlap=200, length_function=len)
chunks = text_splitter.split_text(text)
print(len(chunks))

132


# Retriever and Generator intialization step.

In [9]:
#Initiate Retriever
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distilbert-base-nli-stsb-mean-tokens",model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True})
docs = [Document(page_content=clean_text(chunk)) for chunk in chunks]
doc_search = FAISS.from_documents(docs,embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#initialize Generator
generator_model= GPT2LMHeadModel.from_pretrained('distilgpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
print("Succesfully imported!")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Succesfully imported!


In [11]:
#Retreiving the similar documents from the vectorstore
query= "Can I add another driver to my policy? "
retrieved_docs = doc_search.similarity_search(query)
print(retrieved_docs[0].page_content)

from allowing someone to drive. used, for example if you change from social,  Youre not renting your car out or using a domestic and pleasure use to business use. peertopeer hire scheme to do so.  There are any modifications made to your car see Modifications to your car If youre not sure whether well cover your car if on page 36. its used as part of a car sharing arrangement, please get in touch.  You want to add another driver to the policy, or make any other change to who can drive When we can act on your behalf your car. Were entitled to do either of the following  You want to change to a higher level of cover,  Take over and carry out the negotiation, for example from Third Party, Fire and Theft to defence or settlement of any claim in your Comprehensive. name, or in the name of any other person After your cover starts covered by this policy.  Start legal proceedings in your name, or in You must tell us as soon as possible if


In [12]:
#generate the response for given query
def generate_response(query,context,generator_model,tokenizer,max_input_length=1024, max_output_length=150):
    retrieved_docs = doc_search.similarity_search(query,k=3)
    context = "".join([doc.page_content for doc in retrieved_docs])
    #prompt_template = f"<context> {context}<context> question: {query} </s>"
    prompt_template="""
    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    {context}
    Question:{question}

    Helpful Answers:
    """
    prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])
    #print(input_text)
    #print("------------------------------------------")
    input_text = prompt_template.format(question=query, context=context)
    input_ids = tokenizer.encode(input_text,return_tensors='pt',max_length=max_input_length,truncation=True)
    outputs = generator_model.generate(input_ids,max_length=512,max_new_tokens=256, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return response

query = "What is the procedure of claim? "
llm = HuggingFacePipeline(pipeline=generate_response)
#qa = RetrievalQA(llm=llm, retriever=doc_search.as_retriever())
response = generate_response(query,doc_search,generator_model,tokenizer)
print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    If theres an accident covered by this policy, we may pay reasonable legal costs or expenses to If youre on defend or represent you or any driver covered Third Party, Fire and Theft by this policy Essentials  At a coroners inquest or fatal accident inquiry. Comprehensive  In criminal proceedings to do with the accident. Comprehensive Plus Its up to us whether we do this. If we do, the cover in this section depends on we must agree to all legal costs or expenses the type of insurance you have. beforehand in writing. If we agree to pay these legal costs or expenses, well tell you how much were willing to cover. If you cause an accident If youre deemed at fault for an incident and the Included with other party have solicitors involved, we may need to pay the claimants costs as part of the claim. TPFT Essential Comp Comp Well provide cover if youre found 

In [13]:
#Confirm the response
response.split('Helpful Answers:')[1]

"\n     The procedure is a simple one. You can choose a person or person to represent your case. The person is not a lawyer. It is just a matter of how you can represent the case, how the person can be represented. the lawyer is an attorney. it is simply a question of what you are doing. if your lawyer has a problem with a case or you want to help you get the help of a solicitor, then you will have the right to ask the solicitor to give you the advice you need. you may have an issue with this but you dont have any problem. your solicitor will be able to tell the judge what the problem is. what is your problem? what are your problems? if they are not your fault. they will not be your issue. so if there is any problems, it will take you to a legal lawyer and you won't have it. a professional lawyer will help. in the process of getting the information you needed. for example, a law lawyer can help if a client has an problem or a dispute. this is what they need for a good lawyer to know. a

In [14]:
#load created test csv file
test_df = pd.read_csv('test.csv')
test_df.head()

,Question,Response
0,How much will you pay if my car is damaged?,Where damage to your car is covered under your...
1,Who is covered to drive other cars?,Your certificate of motor insurance will show ...
2,Am I covered if I leave my car unlocked or the...,We won’t pay a claim for theft or attempted th...
3,What’s not included in my cover?,We don’t cover things like: Mechanical or elec...
4,Does Churchill have approved repairers?,Churchill customers have access to a national ...


In [15]:
#generated responsed for first five questions using generate_response function
for q in test_df['Question'][:5]:
  response = generate_response(q,doc_search,generator_model,tokenizer)
  print(response)
  print("---------------------------------------------------------------")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation


    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    Essential Comp Comp Comp Comp If your car is accidentally damaged, we can If your car is damaged, well also replace any choose to either damaged removable electronic equipment,  Repair  well repair the damage ourselves as long as we havent sent a payment to cover or pay to repair it. replacing your car.  Replace  well replace whatever is lost or damaged, if thats more costeffective. Misfuelling using the wrong fuel  Repay  well settle your claim by sending Included with a payment. Comp Comp How it works on page 6 tells you how we Well cover any damage to your car caused by manage repairs and replacements. misfuelling. Youre not covered for Child car seats 8 Included with We wont cover the cost of draining, flushing or replacing the fuel if the wrong fuel is put Essential Comp Comp in your car. If you have a child car seat fitted to your car and your 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    see if the main driver is covered for liability to driving another car. others when driving another car. 8 Well only cover you if you still have your car and it hasnt been written off. Payments made outside the terms Other exclusions 8 of the policy We dont cover  Damage caused by any driver insured on If we have to make a payment that isnt covered this policy to any property they own or are by this policy because were required to do so responsible for. under any countrys laws, we may ask you or  Liability for loss of, or accidental damage the person who is legally responsible to pay us back any payment made that isnt covered to, any car youre driving or any trailer or vehicle youre towing. by this policy. This includes any amount that we have to pay because you dont provide  Death of, or injury to anyone working with accurate and complete informatio

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    the whole journey. This includes driving to and if your car is left from a car park, railway station or bus stop as  Unlocked. part of your journey to and from a permanent  With keys or key fobs in, on, or attached to place of work. the car.  With the engine running. Can I use my car abroad?  With a window or roof open. If you want to use your car abroad, your cover depends on the type of policy you have and Whats not included in my cover? where youre driving. You can find full details We dont cover things like in Where you can drive on page 31.  Mechanical or electrical failure. You may need a Green Card if youre travelling  Wear and tear. abroad. If you need one, please get in touch  Damage to tyres caused by braking, before you travel. We also recommend you take punctures, cuts or bursts. a European Accident Statement with you. You can  Breakdowns

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



    Use the following piece of context to answer the question asked.
    Please try to provide the answer only based on the context

    cancellation date may be different to the cancellation date of the insurance policy. Page 38How the policy works If youve made a claim, andor used your Green  You can find the administration fee Flag cover, before the cancellation or removal mentioned above on your car insurance happens details. For the policy and car insurance cover options,  If you decide to cancel the policy, well give e.g. Protected No Claim Discount you proof of any No Claim Discount. This will  We will not refund any car insurance include any reduction due to claims youve made while insured with us. See No Claim premium if you have made a car insurance Discount NCD on page 40 for more about claim or if one has been made against you this. Other insurers may ask for this proof. during the period of cover regardless of whether you pay annually or by monthly  If you live in Norther

In [16]:
#save contexts for all questions in test data
contexts = []
for q in test_df['Question']:
  retrieved_docs = doc_search.similarity_search(q,k=3)
  context = "".join([doc.page_content for doc in retrieved_docs])
  contexts.append([context])

In [17]:
#Save generated response to check evaluation metrics
generated_response = []
for q in test_df['Question']:
  response = generate_response(q,doc_search,generator_model,tokenizer)
  generated_response.append(response.split('Helpful Answers:')[1].strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation

# Evaluation Metrics

In [19]:
#Save actual responses of in ground truth list
ground_truths = [[x] for x in test_df['Response']]

In [20]:
#calculate rouge score for generated responses
from rouge_score import rouge_scorer
import numpy as np
import sacrebleu
def calculate_rouge(responses,references):
  rs = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
  rouge_scores = {"rouge1": [], "rougeL": []}
  bleu_scores = []
  #bert_scores = {"precision": [], "recall": [], "f1": []}
  for res,ref in zip(responses,references):
    scores = rs.score(res, ref)
    rouge_scores["rouge1"].append(scores["rouge1"].fmeasure)
    rouge_scores["rougeL"].append(scores["rougeL"].fmeasure)
    bleu_score = sacrebleu.sentence_bleu(res, [ref])
    bleu_scores.append(bleu_score.score)
  rouge1 = np.mean(rouge_scores["rouge1"])
  rougeL = np.mean(rouge_scores["rougeL"])
  bleu_score = np.mean(bleu_scores)
  #print(f"Rouge1: {rouge1}, RougeL: {rougeL}, Bleu: {bleu_score}"
  return rouge1,rougeL,bleu_score

calculate_rouge(generated_response,test_df['Response'])

(0.1821405483231928, 0.11734209147165207, 1.0235825476701923)

In [21]:
data = {"question": test_df['Question'],
    "answer": generated_response,
        'contexts':contexts,
    "reference": ground_truths
}
dataset = Dataset.from_dict(data)

In [22]:
# Initialize a sentence transformer model for calculating similarity
similarity_model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

def calculate_relevancy(generated_responses, reference_responses):
    relevancy_scores = []
    for gen, ref in zip(generated_responses, reference_responses):
        embeddings = similarity_model.encode([gen, ref])
        score = util.pytorch_cos_sim(embeddings[0], embeddings[1]).item()
        relevancy_scores.append(score)
    return relevancy_scores

def calculate_faithfulness(generated_responses, reference_responses):
    # Simple heuristic: count the number of reference words in the generated response
    faithfulness_scores = []
    for gen, ref in zip(generated_responses, reference_responses):
        ref_words = set(ref.split())
        gen_words = set(gen.split())
        common_words = ref_words.intersection(gen_words)
        score = len(common_words) / len(ref_words) if ref_words else 0
        faithfulness_scores.append(score)
    return faithfulness_scores

# Calculate faithfulness and relevancy scores
relevancy_scores = calculate_relevancy(generated_response, test_df['Response'])
faithfulness_scores = calculate_faithfulness(generated_response, test_df['Response'])

# Add scores to the dataframe
test_df['Relevancy_Score'] = relevancy_scores
test_df['Faithfulness_Score'] = faithfulness_scores

# Calculate average scores
average_relevancy = sum(relevancy_scores) / len(relevancy_scores)
average_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

print(f"Average Relevancy Score: {average_relevancy}")
print(f"Average Faithfulness Score: {average_faithfulness}")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average Relevancy Score: 0.365368811834243
Average Faithfulness Score: 0.2859834625727287


# Try to Improved Model

In [23]:
#Try to improve model performance by changing prompt template
prompt_template = """
Use the following context to answer the question. Ensure that your answer is concise and directly addresses the question.

Context: {context}

Question: {question}

Answer:
"""


In [24]:
#again same generate function is used
def generate_response(query,context,generator_model,tokenizer,max_input_length=1024, max_output_length=150):
    retrieved_docs = doc_search.similarity_search(query)
    context = "".join([doc.page_content for doc in retrieved_docs])
    #prompt_template = f"<context> {context}<context> question: {query} </s>"
    prompt_template="""
    Use the following context to answer the question. Ensure that your answer is concise and directly addresses the question.

    Context: {context}

    Question: {question}

    Answer:
    """
    prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])
    #print(input_text)
    #print("------------------------------------------")
    input_text = prompt_template.format(question=query, context=context)
    input_ids = tokenizer.encode(input_text,return_tensors='pt',max_length=max_input_length,truncation=True)
    outputs = generator_model.generate(input_ids,max_length=512,max_new_tokens=256, num_return_sequences=1, no_repeat_ngram_size=2)
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return response

query = "What is the procedure of claim? "
llm = HuggingFacePipeline(pipeline=generate_response)
#qa = RetrievalQA(llm=llm, retriever=doc_search.as_retriever())
response = generate_response(query,doc_search,generator_model,tokenizer)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



    Use the following context to answer the question. Ensure that your answer is concise and directly addresses the question.

    Context: If theres an accident covered by this policy, we may pay reasonable legal costs or expenses to If youre on defend or represent you or any driver covered Third Party, Fire and Theft by this policy Essentials  At a coroners inquest or fatal accident inquiry. Comprehensive  In criminal proceedings to do with the accident. Comprehensive Plus Its up to us whether we do this. If we do, the cover in this section depends on we must agree to all legal costs or expenses the type of insurance you have. beforehand in writing. If we agree to pay these legal costs or expenses, well tell you how much were willing to cover. If you cause an accident If youre deemed at fault for an incident and the Included with other party have solicitors involved, we may need to pay the claimants costs as part of the claim. TPFT Essential Comp Comp Well provide cover if youre fou

In [25]:
generated_response = []
for q in test_df['Question']:
  response = generate_response(q,doc_search,generator_model,tokenizer)
  generated_response.append(response.split('Answer:')[1].strip())

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation

In [26]:
# Calculate faithfulness and relevancy scores
relevancy_scores = calculate_relevancy(generated_response, test_df['Response'])
faithfulness_scores = calculate_faithfulness(generated_response, test_df['Response'])

# Add scores to the dataframe
test_df['Relevancy_Score'] = relevancy_scores
test_df['Faithfulness_Score'] = faithfulness_scores

# Calculate average scores
average_relevancy = sum(relevancy_scores) / len(relevancy_scores)
average_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

print(f"Average Relevancy Score: {average_relevancy}")
print(f"Average Faithfulness Score: {average_faithfulness}")


Average Relevancy Score: 0.4170428711560465
Average Faithfulness Score: 0.3235505866088602


# Creating pipelines for transformers

In [65]:
generator_model = GPT2LMHeadModel.from_pretrained('distilgpt2')
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
llm = pipeline(task='text-generation',model=generator_model,tokenizer=tokenizer, max_length=1024)

In [107]:
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Use the following context to answer the question as accurately and concisely as possible.

    Context: {context}
    Question: {question}

    Answer:
    """
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
        (
            "user",
            f"{prompt_template}",
        ),
    ]
)


In [110]:
#Intialize the retriver
retriever = doc_search.as_retriever(search_type="similarity",search_kwargs={"k":3})

In [121]:
#Initializing hugging face pipeline
hf_pipeline = HuggingFacePipeline(pipeline=llm,model_kwargs={'device':'cpu','max_new_tokens': 300,"temperature": 0})

In [122]:
#create a chain of retriever and generator
qa_chain = load_qa_chain(llm=hf_pipeline, chain_type="stuff", prompt=prompt_template)
qa = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=qa_chain,
    return_source_documents=True,
)

In [123]:
#generate response
def generate_response(query):
    result = qa({"query": query})
    return result["result"]
    #return response

query = "What is the procedure of claim? "
response = generate_response(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



    Use the following context to answer the question as accurately and concisely as possible. 

    Context: If theres an accident covered by this policy, we may pay reasonable legal costs or expenses to If youre on defend or represent you or any driver covered Third Party, Fire and Theft by this policy Essentials  At a coroners inquest or fatal accident inquiry. Comprehensive  In criminal proceedings to do with the accident. Comprehensive Plus Its up to us whether we do this. If we do, the cover in this section depends on we must agree to all legal costs or expenses the type of insurance you have. beforehand in writing. If we agree to pay these legal costs or expenses, well tell you how much were willing to cover. If you cause an accident If youre deemed at fault for an incident and the Included with other party have solicitors involved, we may need to pay the claimants costs as part of the claim. TPFT Essential Comp Comp Well provide cover if youre found to be legally Cover for othe

In [124]:
#generate response for all test data
generated_response = []
for q in test_df['Question']:
  response = generate_response(q)
  generated_response.append(response.split('Answer:')[1].strip())

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [125]:
#calculating metrices
relevancy_scores = calculate_relevancy(generated_response, test_df['Response'])
faithfulness_scores = calculate_faithfulness(generated_response, test_df['Response'])
average_relevancy = sum(relevancy_scores) / len(relevancy_scores)
average_faithfulness = sum(faithfulness_scores) / len(faithfulness_scores)

print(f"Average Relevancy Score: {average_relevancy}")
print(f"Average Faithfulness Score: {average_faithfulness}")


Average Relevancy Score: 0.42862514839057
Average Faithfulness Score: 0.33886070564558307
